In [1]:
%load_ext autoreload
%autoreload 2
%load_ext nb_black
%load_ext lab_black

<IPython.core.display.Javascript object>

In [2]:
# default_exp model

<IPython.core.display.Javascript object>

# Model

In [3]:
# hide
from nbdev.showdoc import *

<IPython.core.display.Javascript object>

In [4]:
#export
import gc
import uuid
import joblib
import pickle
import numpy as np
import pandas as pd
import lightgbm as lgb
from pathlib import Path
from typing import Union
from tqdm.auto import tqdm
from catboost import CatBoost
from typeguard import typechecked
from abc import ABC, abstractmethod
from sklearn.dummy import DummyRegressor

from numerai_blocks.dataset import NumerFrame, create_numerframe
from numerai_blocks.preprocessing import display_processor_info

<IPython.core.display.Javascript object>

## 0. Base

### 0.1. BaseModel

The `BaseModel` is an abstract base class where the `predict` method is central. All models should inherit from `BaseModel` and be sure to implement the `predict` method.

In general, models make use of files that are loaded in. Therefore, the correct `file_suffix` of the models in question should be specified. If no model files are involved in your model you should pass an empty string `""` to the `file_suffix` parameter.

In [5]:
#export
class BaseModel(ABC):
    """
    Setup for model prediction on a Dataset.

    :param model_directory: Main directory from which to read in models.
    :param model_name: Name that will be used to create column names and for display purposes.
    """
    def __init__(self, model_directory: str, model_name: str = None, *args, **kwargs):
        self.model_directory = Path(model_directory)
        self.__dict__.update(*args, **kwargs)
        self.model_name = model_name if model_name else uuid.uuid4().hex
        self.prediction_col_name = f"prediction_{self.model_name}"
        self.description = f"{self.__class__.__name__}: '{self.model_name}' prediction"

    @abstractmethod
    def predict(self, dataf: Union[pd.DataFrame, NumerFrame]) -> NumerFrame:
        """ Return NumerFrame with column added for prediction. """
        ...
        return NumerFrame(dataf)

    def __call__(self, dataf: Union[pd.DataFrame, NumerFrame]) -> NumerFrame:
        return self.predict(dataf=dataf)

<IPython.core.display.Javascript object>

### 0.2. BaseModel

A `DirectoryModel` assumes that you have a directory of models and you want to load + predict for all models with a certain `file_suffix` (for example, `.joblib`, `.cbm` or `.lgb`). This class load in all models with the suffix and makes predictions for them.

When inheriting from `DirectoryModel` the only method that needs to be implemented is `_load_models`. It should instantiate all models and return them as a list.

In [6]:
#export
class DirectoryModel(BaseModel):
    """
    Base class implementation for JoblibModel, CatBoostModel, LGBMModel, etc.
    Walks through every file with given file_suffix in a directory.
    :param model_directory: Main directory from which to read in models.
    :param file_suffix: File format to load (For example, .joblib, .cbm or .lgb)
    :param model_name: Name that will be used to create column names and for display purposes.
    """
    def __init__(self, model_directory: str, file_suffix: str, model_name: str = None, *args, **kwargs):
        super().__init__(model_directory=model_directory,
                         model_name=model_name,
                         *args, **kwargs
                         )
        self.file_suffix = file_suffix
        self.model_paths = list(self.model_directory.glob(f'*.{self.file_suffix}'))
        if self.file_suffix:
            assert self.model_paths, f"No {self.file_suffix} files found in {self.model_directory}."
        self.total_models = len(self.model_paths)

    @display_processor_info
    def predict(self, dataf: Union[pd.DataFrame, NumerFrame], *args, **kwargs) -> NumerFrame:
        """
        Use all recognized models to make predictions and average them out.
        :param dataf: A Preprocessed DataFrame where all its features can be passed to the model predict method.
        *args, **kwargs will be parsed into the model.predict method.
        :return: A new dataset with prediction column added.
        """
        dataf.loc[:, self.prediction_col_name] = np.zeros(len(dataf))
        models = self.load_models()
        for model in tqdm(models, desc=self.description, position=1):
            predictions = model.predict(dataf.get_feature_data, *args, **kwargs)
            dataf.loc[:, self.prediction_col_name] += predictions / self.total_models
        del models; gc.collect()
        return NumerFrame(dataf)

    @abstractmethod
    def load_models(self) -> list:
        """ Instantiate all models detected in self.model_paths. """
        ...


<IPython.core.display.Javascript object>

## 1. Standard model formats

### 1.1. Single Model file

In many cases you would just want to load a single model file and create prediction for that model.

This class supports multiple model formats for easy use. All models should have a `.predict` method and currently `.joblib`, `.cbm`, `.lgb`, `.pkl` and `.pickle` format are supported. For XGBoost models we recommend saving them as `.joblib`.

In [7]:
#export
@typechecked
class SingleModel(BaseModel):
    def __init__(self, model_file_path: str, model_name: str = None, *args, **kwargs):
        self.model_file_path = Path(model_file_path)
        assert self.model_file_path.exists(), f"File path '{self.model_file_path}' does not exist."
        assert self.model_file_path.is_file(), f"File path must point to file. Not valid for '{self.model_file_path}'."
        super().__init__(model_directory=str(self.model_file_path.parent),
                         model_name=model_name,
                         *args, **kwargs
                         )
        self.model_suffix = self.model_file_path.suffix
        self.suffix_to_model_mapping = {".joblib": joblib.load,
                                        ".cbm": CatBoost().load_model,
                                        ".pkl": pickle.load,
                                        ".pickle": pickle.load}
        self.__check_valid_suffix()

    def predict(self, dataf: Union[pd.DataFrame, NumerFrame], *args, **kwargs) -> NumerFrame:
        model = self._load_model()
        predictions = model.predict(dataf.get_feature_data, *args, **kwargs)
        prediction_cols = self.get_prediction_col_names(predictions.shape)
        dataf.loc[:, prediction_cols] = predictions
        del model; gc.collect()
        return NumerFrame(dataf)

    def _load_model(self, *args, **kwargs):
        """ Load arbitrary model from path using suffix to model mapping. """
        return self.suffix_to_model_mapping[self.model_suffix](str(self.model_file_path), *args, **kwargs)

    def get_prediction_col_names(self, pred_shape: tuple) -> list:
        """ Create multiple columns if predictions are multi-target. """
        if len(pred_shape) > 1:
            # Multi target
            prediction_cols = [f"{self.prediction_col_name}_{i}" for i in range(pred_shape[1])]
        else:
            # Single target
            prediction_cols = [self.prediction_col_name]
        return prediction_cols

    def __check_valid_suffix(self):
        """ Detailed message if model is not supported in this class. """
        try:
            self.suffix_to_model_mapping[self.model_suffix]
        except KeyError:
            raise NotImplementedError(
                f"Format '{self.model_suffix}' is not available. Available versions are {list(self.suffix_to_model_mapping.keys())}"
            )


<IPython.core.display.Javascript object>

In [8]:
dataset = create_numerframe("test_assets/mini_numerai_version_2_data.parquet")
test_paths = ["test_assets/joblib_v2_example_model.joblib"]
for path in test_paths:
    model = SingleModel(path, model_name="test")
    print(model.predict(dataset).get_prediction_data.head(2))

                  prediction_test
id                               
n559bd06a8861222         0.506948
n9d39dea58c9e3cf         0.492578


<IPython.core.display.Javascript object>

In [9]:
model = SingleModel(test_paths[0], model_name="test")
model.suffix_to_model_mapping

{'.joblib': <function joblib.numpy_pickle.load(filename, mmap_mode=None)>,
 '.cbm': <bound method CatBoost.load_model of <catboost.core.CatBoost object at 0x7fbdc8d430d0>>,
 '.pkl': <function _pickle.load(file, *, fix_imports=True, encoding='ASCII', errors='strict')>,
 '.pickle': <function _pickle.load(file, *, fix_imports=True, encoding='ASCII', errors='strict')>}

<IPython.core.display.Javascript object>

### 1.2. Joblib directory

Many frameworks like `sklearn` models can conveniently be saved as `.joblib` files. This class automatically load all `.joblib` files in a given folders and generates (averaged out) predictions.

In [10]:
#export
@typechecked
class JoblibModel(DirectoryModel):
    """ Load and predict for arbitrary models saved as .joblib. """
    def __init__(self, model_directory: str, model_name: str = None, *args, **kwargs):
        file_suffix = 'joblib'
        super().__init__(model_directory=model_directory,
                         file_suffix=file_suffix,
                         model_name=model_name,
                         *args, **kwargs
                         )

    def load_models(self) -> list:
        return [joblib.load(path) for path in self.model_paths]

<IPython.core.display.Javascript object>

In [11]:
dataset = create_numerframe("test_assets/mini_numerai_version_2_data.parquet", metadata={"version": 2})
model = JoblibModel("test_assets", model_name="Joblib_LGB")
predictions = model.predict(dataset).get_prediction_data
assert predictions['prediction_Joblib_LGB'].between(0, 1).all()
predictions.head(3)

JoblibModel: 'Joblib_LGB' prediction:   0%|          | 0/1 [00:00<?, ?it/s]

✅ Finished step DirectoryModel. Output shape=(10, 1074). Time taken for step: 
0:00:00.138489. ✅

,prediction_Joblib_LGB
id,
n559bd06a8861222,0.506948
n9d39dea58c9e3cf,0.492578
nb64f06d3a9fc9f1,0.490879


<IPython.core.display.Javascript object>

### 1.3. Catboost directory (.cbm)

This model setup loads in all `CatBoost` (`.cbm`) models present in a given directory and makes (averaged out) predictions.

In [12]:
#export
@typechecked
class CatBoostModel(DirectoryModel):
    """ Load and predict with all .cbm models (CatBoostRegressor) in directory. """
    def __init__(self, model_directory: str, model_name: str = None, *args, **kwargs):
        file_suffix = 'cbm'
        super().__init__(model_directory=model_directory,
                         file_suffix=file_suffix,
                         model_name=model_name,
                         *args, **kwargs
                         )

    def load_models(self) -> list:
        return [CatBoost().load_model(path) for path in self.model_paths]

<IPython.core.display.Javascript object>

In [13]:
from numerai_blocks.preprocessing import GroupStatsPreProcessor
dataset = create_numerframe("test_assets/mini_numerai_version_1_data.csv", {"version": 1})
processed_dataset = GroupStatsPreProcessor()(dataset)
model = CatBoostModel("test_assets", model_name="CB")

✅ Finished step GroupStatsPreProcessor. Output shape=(10, 332). Time taken for step: 
0:00:00.022835. ✅

<IPython.core.display.Javascript object>

In [14]:
predictions = model.predict(processed_dataset).get_prediction_data
assert predictions['prediction_CB'].between(0, 1).all()
predictions.head(3)

CatBoostModel: 'CB' prediction:   0%|          | 0/1 [00:00<?, ?it/s]

✅ Finished step DirectoryModel. Output shape=(10, 333). Time taken for step: 0:00:00.095464.
✅

,prediction_CB
0,0.492046
1,0.499881
2,0.485325


<IPython.core.display.Javascript object>

### 1.4. LightGBM (.lgb)

This model setup loads in all `LightGBM` (`.lgb`) models present in a given directory and makes (averaged out) predictions.

In [15]:
#export
@typechecked
class LGBMModel(DirectoryModel):
    """ Load and predict with all .lgb models (LightGBM) in directory. """
    def __init__(self, model_directory: str, model_name: str = None, *args, **kwargs):
        file_suffix = 'lgb'
        super().__init__(model_directory=model_directory,
                         file_suffix=file_suffix,
                         model_name=model_name,
                         *args, **kwargs
                         )

    def load_models(self) -> list:
        return [lgb.Booster(model_file=str(path)) for path in self.model_paths]

<IPython.core.display.Javascript object>

In [16]:
dataset = create_numerframe("test_assets/mini_numerai_version_2_data.parquet")
model = LGBMModel("test_assets", model_name="LGB")
predictions = model.predict(dataset).get_prediction_data
assert predictions['prediction_LGB'].between(0, 1).all()
predictions.head(3)

LGBMModel: 'LGB' prediction:   0%|          | 0/1 [00:00<?, ?it/s]

✅ Finished step DirectoryModel. Output shape=(10, 1074). Time taken for step: 
0:00:00.142040. ✅

,prediction_LGB
id,
n559bd06a8861222,0.506948
n9d39dea58c9e3cf,0.492578
nb64f06d3a9fc9f1,0.490879


<IPython.core.display.Javascript object>

## 2. Baseline models

Setting a baseline is always an important step when modeling new data. Therefore, we have implemented a few different baselines.

### 2.1. ConstantModel

This model simply outputs a constant of your choice. Great for setting classification baselines.

In [17]:
#export
class ConstantModel(BaseModel):
    """
    WARNING: Only use this Model for testing purposes.
    Create constant prediction.
    """
    def __init__(self, constant: float = 0.5, model_name: str = None):
        self.constant = constant
        model_name = model_name if model_name else f"constant_{self.constant}"
        super().__init__(model_directory="",
                         model_name=model_name
                         )
        self.clf = DummyRegressor(strategy='constant', constant=constant).fit([0.], [0.])

    def predict(self, dataf: Union[pd.DataFrame, NumerFrame]) -> NumerFrame:
        dataf.loc[:, self.prediction_col_name] = self.clf.predict(dataf.get_feature_data)
        return NumerFrame(dataf)

<IPython.core.display.Javascript object>

In [18]:
constant = 0.85
dataset = create_numerframe("test_assets/mini_numerai_version_1_data.csv")
constant_model = ConstantModel(constant=constant)
predictions = constant_model.predict(dataset).get_prediction_data
assert (predictions.to_numpy() == constant).all()
predictions.head(3)

,prediction_constant_0.85
0,0.85
1,0.85
2,0.85


<IPython.core.display.Javascript object>

### 2.2. RandomModel

This model returns uniformly distributed predictions (range $[0...1)$). Great baseline for evaluating regression models against.

In [19]:
#export
class RandomModel(BaseModel):
    """
    WARNING: Only use this Model for testing purposes.
    Create uniformly distributed predictions.
    """
    def __init__(self, model_name: str = None):
        model_name = model_name if model_name else "random"
        super().__init__(model_directory="",
                         model_name=model_name
                         )

    def predict(self, dataf: Union[pd.DataFrame, NumerFrame]) -> NumerFrame:
        dataf.loc[:, self.prediction_col_name] = np.random.uniform(size=len(dataf))
        return NumerFrame(dataf)

<IPython.core.display.Javascript object>

In [20]:
dataset = create_numerframe("test_assets/mini_numerai_version_1_data.csv", metadata={})
random_model = RandomModel()
predictions = random_model.predict(dataset).get_prediction_data
assert predictions['prediction_random'].between(0, 1).all()
predictions.head(3)

,prediction_random
0,0.813524
1,0.043787
2,0.925226


<IPython.core.display.Javascript object>

## 3. Custom Model

### 3.1. From BaseModel

Arbitrary models can be instantiated by inheriting from `BaseModel` and defining the `file_suffix`. All model_paths of models that are in `model_directory` will be defined upon instantiation. Arbitrary logic (model loading, prediction, etc.) can be defined in `.predict` as long as the method takes a `NumerFrame` or `DataFrame` as input and outputs a `NumerFrame`. The Model should be able to typecheck by adding the `@typeguard.typechecked` decorator at the top of the class.

For clear console output we recommend adding the `@display_processor_info` decorator to the `predict` method.

In [21]:
#export
@typechecked
class AwesomeModel(BaseModel):
    """
    - TEMPLATE -
    Predict with arbitrary model formats.
    """
    def __init__(self, model_directory: str, model_name: str = None, *args, **kwargs):
        super().__init__(model_directory=model_directory,
                         model_name=model_name,
                         *args, **kwargs
                         )

    @display_processor_info
    def predict(self, dataf: Union[pd.DataFrame, NumerFrame]) -> NumerFrame:
        """ Return Dataset with column(s) added for prediction(s). """
        # NumerFrame functionality to get all features
        feature_df = dataf.get_feature_data
        # Predict and add to new column
        ...
        # Parse all contents of Dataset to the next pipeline step
        return NumerFrame(dataf)

<IPython.core.display.Javascript object>

### 3.2. From DirectoryModel

You also may want to implement a setup similar to `JoblibModel` and `CatBoostModel`, where you want to load in all models of a certain type from a directory and predict for all. The model should also have a `.predict` method. If this sounds like your use case, inherit from `DirectoryModel` and be sure to implement the `load_models` method.

For an `DirectoryModel` you should also specify a `file_suffix` (like joblib or cbm) which will be used to load in the models from a directory.

The `.predict` method will in this case already be implemented, but can be overridden if the prediction logic is more complex. For example, if you want to apply weighted averaging or geometric mean for models within a given directory.

In [22]:
#export
@typechecked
class AwesomeDirectoryModel(DirectoryModel):
    """
    - TEMPLATE -
    Load in all models of arbitrary file format where model has .predict method.
    """
    def __init__(self, model_directory: str, model_name: str = None, *args, **kwargs):
        file_suffix = 'anything'
        super().__init__(model_directory=model_directory,
                         file_suffix=file_suffix,
                         model_name=model_name,
                         *args, **kwargs
                         )

    def load_models(self) -> list:
        """ Instantiate all models and return as a list. (abstract method) """
        ...

<IPython.core.display.Javascript object>

----------------------------------------------

In [23]:
# hide
# Run this cell to sync all changes with library
from nbdev.export import notebook2script

notebook2script()

Converted 01_download.ipynb.
Converted 02_numerframe.ipynb.
Converted 03_preprocessing.ipynb.
Converted 04_model.ipynb.
Converted 05_postprocessing.ipynb.
Converted 06_modelpipeline.ipynb.
Converted 07_evaluation.ipynb.
Converted 08_key.ipynb.
Converted 09_submission.ipynb.
Converted 10_staking.ipynb.
Converted index.ipynb.


<IPython.core.display.Javascript object>